<a href="https://colab.research.google.com/github/GomideB/ExemploDoc/blob/main/OFICIAL_SOS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instalação das bibliotecas necessárias (apenas uma vez)
# Usando a instalação robusta que funcionou para as dependências básicas
# --- Seção de Configuração Inicial e Instalação de Dependências ---
# Esta seção é crucial para preparar o ambiente do Google Colab.
# Ela garante que todas as bibliotecas necessárias para o Agente de IA e a interação com as APIs do Google
# sejam instaladas e configuradas corretamente.

# Instalação de bibliotecas:
# O comando '%pip' é específico do ambiente de notebooks (como Google Colab)
# e é equivalente a 'pip install'.
# '-q': Modo silencioso, suprime a maioria das mensagens de saída.
# '--upgrade': Atualiza as bibliotecas para as versões mais recentes.
# '--force-reinstall': Força a reinstalação, útil para garantir que as versões corretas sejam carregadas
# após possíveis conflitos ou instalações anteriores.
# 'google-genai': Biblioteca oficial para interagir com o modelo Gemini (Google AI Studio).
# 'google-adk': Google Agent Development Kit, um framework para construir e gerenciar agentes de IA.
# 'aiohttp': Biblioteca para fazer requisições HTTP assíncronas, utilizada para interagir com a API do Google Maps.
%pip -q install --upgrade --force-reinstall google-genai google-adk aiohttp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 624.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.3/171.3 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 74.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13

In [2]:
# Configuração de Logs:
# A biblioteca 'asyncio' (usada para operações assíncronas) pode gerar muitos avisos de logs.
# Esta linha silencia esses avisos, tornando a saída do console mais limpa e focada.
import logging
logging.getLogger('asyncio').setLevel(logging.CRITICAL)

In [ ]:
# Reinício do Ambiente de Execução:
# Em muitos casos, após instalar ou atualizar bibliotecas em ambientes como o Colab,
# é necessário reiniciar o kernel para que as novas versões das bibliotecas sejam efetivamente carregadas.
# 'os.kill(os.getpid(), 9)': Este comando encerra o processo Python atual (o kernel do Colab),
# forçando um reinício do ambiente.
# Após isso, o Colab geralmente informa que o ambiente foi reiniciado e você precisa executar
# as próximas células manualmente.
# Importante: Reinicie o ambiente de execução após a instalação para garantir que as novas versões sejam carregadas
import os
os.kill(os.getpid(), 9)

In [ ]:
# Após o Colab reiniciar, execute o restante do código na próxima célula.

# --- Configurações Iniciais ---
# Esta seção é executada após o reinício do ambiente e foca na configuração das chaves de API
# e na importação de todas as bibliotecas necessárias para o funcionamento do agente.
import os
# 'google.colab.userdata': Usado para acessar "Secrets" (segredos/variáveis de ambiente seguras)
# armazenados no Google Colab, garantindo que suas chaves de API não fiquem expostas diretamente no código.
from google.colab import userdata

# SUPRESSÃO DE AVISOS DO ASYNCIO (Repetição para garantir após o restart)
import logging
logging.getLogger('asyncio').setLevel(logging.CRITICAL) # <--- Linha para suprimir os avisos

# Configura as API Keys do Google Gemini e Google Maps
# 'os.environ': Dicionário que representa as variáveis de ambiente do sistema.
# As chaves são recuperadas dos "Secrets" do Colab e definidas como variáveis de ambiente,
# o que é uma prática segura.
# 'GOOGLE_API_KEY': Chave para acessar o modelo Gemini.
# 'Maps_API': Chave para acessar a API do Google Maps.
# Certifique-se de que 'GOOGLE_API_KEY' e 'Maps_API_KEY'
# estão adicionadas nas "Secrets" do Colab (ícone de chave no painel esquerdo)
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
os.environ["GOOGLE_MAPS_API"] = userdata.get('GOOGLE_MAPS_API')
# Atribui a chave do Maps a uma variável local para uso mais fácil nas funções.
Maps_API_KEY = os.environ["GOOGLE_MAPS_API"] # Acessa a chave do env

In [ ]:
# Importações de Bibliotecas:
# 'asyncio': Para programação assíncrona (operações que não bloqueiam o programa enquanto esperam).
# 'aiohttp': Cliente HTTP assíncrono para requisições de rede.
# 'google.genai': Biblioteca principal para interagir com o modelo Gemini.
# 'google.genai.types': Tipos de dados específicos para a API Gemini (e.g., Content, Part).
# 'google.adk.agents.Agent': Classe base para definir agentes de IA no ADK.
# 'google.adk.runners.Runner': Usado para executar um agente e gerenciar sua sessão.
# 'google.adk.sessions.InMemorySessionService': Serviço de sessão simples para o ADK, que armazena sessões na memória.
# 'google.adk.tools.Google Search': Uma ferramenta pré-construída do ADK que permite aos agentes fazerem buscas no Google.
# 'typing': Módulo para hints de tipo (e.g., List, Dict, Optional) para melhor legibilidade e verificação de erros.
# 'nest_asyncio': Permite aninhar loops de evento asyncio, essencial para rodar asyncio em ambientes como Jupyter/Colab.
# 'datetime.date': Para lidar com datas (não diretamente usado no fluxo principal, mas comum).
# 'textwrap': Para formatar texto (usado na função 'to_markdown').
# 'IPython.display.display', 'Markdown', 'HTML': Para exibir conteúdo formatado diretamente no notebook.
import asyncio
import aiohttp
import google.genai # Importa a biblioteca genai completa
from google.genai import types
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search # Importa a ferramenta de busca do Google
from typing import List, Dict, Optional
import nest_asyncio
from datetime import date
import textwrap
from IPython.display import display, Markdown, HTML # HTML para exibir URLs clicáveis
import warnings

# Suprime avisos gerais do Python. Use com cautela em produção.
warnings.filterwarnings("ignore")
# Aplica o patch para permitir que loops de evento asyncio sejam aninhados.
# Necessário para evitar erros de 'RuntimeError: This event loop is already running' no Colab.
nest_asyncio.apply() # Essencial para rodar asyncio em notebooks do Colab

# Inicializa o cliente Gemini UMA VEZ
# Cria uma instância do cliente Gemini, que será usada para interagir com os modelos de IA.
client = google.genai.Client()

In [ ]:
# --- Funções Auxiliares Comuns ---
# Estas funções servem como utilitários para interagir com os agentes e formatar a saída.

def executar_agente(agent: Agent, mensagem: str) -> str:
    """
    Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final.
    Usa um 'for' loop normal, pois 'runner.run()' retorna um gerador síncrono.
    """
    # Cria um serviço de sessão na memória para o agente.
    # Isso permite que o agente mantenha o contexto da conversa.
    session_service = InMemorySessionService()
    # Cria uma sessão única para o usuário.
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")

    # Cria um 'Runner' para o agente. O Runner é responsável por orquestrar a execução do agente.
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)

    # Prepara a mensagem do usuário no formato esperado pelo modelo Gemini (tipo Content).
    content = types.Content(role="user", parts=[types.Part(text=mensagem)])
    resposta = ""
    # Itera sobre os eventos gerados pelo Runner.
    # O Runner pode gerar múltiplos eventos (pensamentos do agente, uso de ferramentas, resposta final).
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        # Verifica se o evento é a resposta final do agente.
        if event.is_final_response():
            # Itera sobre as partes da resposta (pode haver múltiplas partes de texto, imagem, etc.).
            for part in event.content.parts:
                # Se a parte for texto, adiciona à resposta final.
                if part.text:
                    resposta += part.text + "\n"
    return resposta

def to_markdown(text):
    """Função auxiliar para exibir texto formatado em Markdown no Colab."""
    # Substitui bullet points comuns por asteriscos para renderização Markdown.
    text = text.replace('•', ' *')
    # Adiciona um "> " (sinal de citação de Markdown) a cada linha do texto
    # para melhor visualização no Colab, simulando uma citação ou bloco destacado.
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
# --- Definição e Instanciação Única dos Agentes ---
# Esta é a parte central do sistema de orquestração de IA, onde cada agente
# é definido com sua função específica (instrução), o modelo de IA a ser usado,
# e uma descrição para o sistema orquestrador.

agente_triagem = Agent(
    name="agente_triagem", # Nome único do agente.
    model="gemini-2.0-flash", # Modelo de IA a ser usado (Flash é mais rápido e econômico).
    instruction="""Você é um assistente de primeiros socorros.
    Pergunte ao usuário sobre o que ele está sentindo para entender a situação de emergência.
    Mantenha a conversa breve e foque em obter uma descrição clara dos sintomas.
    Após a descrição inicial, **pergunte:** "Você sente mais alguma dor ou algo diferente que eu deva saber?"
    """, # A instrução principal do agente, definindo seu comportamento e objetivo.
    description="Agente para coletar os sintomas iniciais do usuário." # Descrição para o sistema orquestrador.
)

# NOVO AGENTE
# Agente para consolidar sintomas adicionais, garantindo que todas as informações
# do usuário sejam agrupadas de forma coerente antes de serem passadas para os próximos agentes.
agente_coleta_sintomas_adicionais = Agent(
    name="agente_coleta_sintomas_adicionais",
    model="gemini-2.0-flash",
    instruction="""Você é um assistente de saúde.
    Sua tarefa é consolidar informações de sintomas. Receberá um resumo dos sintomas iniciais
    e uma descrição de sintomas adicionais fornecidos pelo usuário.
    Crie uma descrição única, clara e completa de TODOS os sintomas do paciente,
    sem incluir nenhuma pergunta ou instrução adicional. Se o usuário indicou que não há
    sintomas adicionais, apenas forneça o resumo dos sintomas iniciais.
    """,
    description="Agente para coletar e consolidar sintomas adicionais do usuário."
)


In [ ]:
agente_busca_primeiros_socorros = Agent(
    name="agente_busca_primeiros_socorros",
    model="gemini-2.0-flash",
    instruction="""Você é um especialista em primeiros socorros.
    Utilize a ferramenta de busca do Google (Google Search) para encontrar
    informações confiáveis e atualizadas sobre como tratar a condição médica
    descrita pelo usuário. Foque em fontes oficiais de saúde, como
    ministérios da saúde, hospitais renomados e organizações de primeiros
    socorros.  Limite a busca a 5 resultados mais relevantes.""",
    description="Agente para buscar informações de primeiros socorros no Google.",
    tools=[google_search] # Este agente tem a capacidade de usar a ferramenta de busca do Google.
)

In [ ]:
agente_planejamento_acoes = Agent(
    name="agente_planejamento_acoes",
    model="gemini-2.0-flash",
    instruction="""Você é um especialista em primeiros socorros.
    Analise os resultados da busca do Google fornecidos e crie um plano
    conciso e lógico de ações de primeiros socorros a serem recomendadas
    ao usuário. O plano deve ser claro, objetivo e priorizar as ações mais
    urgentes. Inclua, se necessário, orientações sobre quando procurar
    ajuda médica profissional e como proceder até lá.""",
    description="Agente para planejar as ações de primeiros socorros."
)

In [ ]:
agente_orientacao_detalhada = Agent(
    name="agente_orientacao_detalhada",
    model="gemini-2.0-flash",
    instruction="""Você é um especialista em primeiros socorros.
    Com base no plano de ações fornecido, gere instruções detalhadas e fáceis
    de seguir para o usuário. Inclua, se possível, links para vídeos do YouTube
    que demonstrem as técnicas de primeiros socorros recomendadas.
    Mencione a importância de procurar ajuda médica profissional e
    oriente o usuário sobre como proceder até lá.""",
    description="Agente para gerar orientações detalhadas de primeiros socorros."
)

In [ ]:
agente_avaliacao_emergencia = Agent(
    name="agente_avaliacao_emergencia",
    model="gemini-2.0-flash",
    instruction="""Você é um especialista em primeiros socorros.
    Analise a descrição dos sintomas do usuário e o plano de ações de
    primeiros socorros para determinar a gravidade da situação.
    Se a situação for de emergência (e.g., suspeita de infarto,
    dificuldade respiratória grave, perda de consciência), instrua o usuário
    a ligar imediatamente para o serviço de emergência (192 - SAMU).
    Caso contrário, oriente o usuário a procurar atendimento médico
    em um hospital ou clínica, fornecendo o endereço e a rota.""",
    description="Agente para avaliar a gravidade da situação e recomendar a ação apropriada."
)

In [ ]:
agente_localizacao_ajuda_medica = Agent(
    name="agente_localizacao_ajuda_medica",
    model="gemini-2.0-flash",
    instruction="""Você é um especialista em primeiros socorros.
    Com base na localização fornecida, você pode raciocinar sobre a necessidade de hospitais e médicos.
    No entanto, a busca real e a geração de rotas serão feitas por funções auxiliares, que te entregarão os dados.""",
    description="Agente para raciocinar sobre a necessidade de localizar ajuda médica."
)

In [ ]:
agente_rota_navegacao = Agent(
    name="agente_rota_navegacao",
    model="gemini-2.0-flash",
    instruction="""Você é um especialista em primeiros socorros.
    Com base nos hospitais e na sua localização, você pode raciocinar sobre a necessidade de rotas.
    No entanto, a geração de links de rotas será feita por funções auxiliares, que te entregarão os dados.""",
    description="Agente para raciocinar sobre rotas para hospitais."
)

In [ ]:
# --- Funções Auxiliares para a API do Google Maps (Permanecem assíncronas) ---
# Estas funções são responsáveis por interagir diretamente com a API do Google Maps,
# utilizando a chave de API e a biblioteca aiohttp para requisições assíncronas.
# Elas buscam informações sobre hospitais e médicos próximos e geram links de rotas.

async def buscar_hospitais_proximos(latitude: float, longitude: float, raio: int = 50) -> List[Dict]:
    """
    Busca hospitais próximos a uma dada localização usando a API Google Places Nearby Search.
    :param latitude: Latitude da localização de busca.
    :param longitude: Longitude da localização de busca.
    :param raio: Raio de busca em KM (convertido para metros na URL).
    :return: Lista de dicionários com nome, endereço, localização e avaliação dos hospitais.
    """
    # CORRIGIDO: Usando Maps_API_KEY corretamente (a variável global Maps_API_KEY).
    # Monta a URL da API Google Places Nearby Search.
    # 'location': Latitude e longitude do ponto central da busca.
    # 'radius': Raio de busca em metros (50 km = 50000 metros).
    # 'type=hospital': Filtra resultados para hospitais.
    # 'key': Sua chave de API do Google Maps.
    # 'language=pt': Define o idioma da resposta para português.
    url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={latitude},{longitude}&radius={raio * 1000}&type=hospital&key={Maps_API_KEY}&language=pt"
    # Cria uma sessão assíncrona HTTP. 'async with' garante que a sessão seja fechada corretamente.
    async with aiohttp.ClientSession() as session:
        try:
            # Faz a requisição GET para a URL.
            async with session.get(url) as response:
                # Verifica se a resposta HTTP teve sucesso (código 2xx). Levanta exceção se não.
                response.raise_for_status()
                # Converte a resposta JSON em um dicionário Python.
                data = await response.json()
                # Verifica o status retornado pela API do Google Places.
                if data["status"] == "OK":
                    hospitais = []
                    # Itera sobre os resultados da busca e extrai as informações relevantes.
                    for result in data["results"]:
                        hospitais.append({
                            "nome": result.get("name"),
                            "endereco": result.get("vicinity"), # 'vicinity' geralmente contém o endereço formatado.
                            "localizacao": result.get("geometry", {}).get("location"),
                            "rating": result.get("rating")
                        })
                    return hospitais
                elif data["status"] == "ZERO_RESULTS":
                    print("Não foram encontrados hospitais na área especificada.")
                    return []
                else:
                    print(f"Erro ao buscar hospitais: {data['status']}")
                    return []
        except aiohttp.ClientError as e:
            # Captura erros relacionados a requisições HTTP (e.g., problemas de rede, status 4xx/5xx).
            print(f"Erro de requisição HTTP: {e}")
            return []
        except Exception as e:
            # Captura quaisquer outros erros inesperados.
            print(f"Erro inesperado: {e}")
            return []

async def buscar_medicos_proximos(latitude: float, longitude: float, raio: int = 50) -> List[Dict]:
    """
    Busca médicos/clínicas próximos a uma dada localização usando a API Google Places Nearby Search.
    Funciona de forma similar à função de busca de hospitais, mas com 'type=doctor'.
    """
    # CORRIGIDO: Usando Maps_API_KEY corretamente
    url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={latitude},{longitude}&radius={raio * 1000}&type=doctor&key={Maps_API_KEY}&language=pt"
    async with aiohttp.ClientSession() as session:
        try:
            async with session.get(url) as response:
                response.raise_for_status()
                data = await response.json()
                if data["status"] == "OK":
                    medicos = []
                    for result in data["results"]:
                        medicos.append({
                            "nome": result.get("name"),
                            "endereco": result.get("vicinity"),
                            "localizacao": result.get("geometry", {}).get("location"),
                            "rating": result.get("rating")
                        })
                    return medicos
                elif data["status"] == "ZERO_RESULTS":
                    print("Não foram encontrados médicos na área especificada.")
                    return []
                else:
                    print(f"Erro ao buscar médicos: {data['status']}")
                    return []
        except aiohttp.ClientError as e:
            print(f"Erro de requisição HTTP: {e}")
            return []
        except Exception as e:
            print(f"Erro inesperado: {e}")
            return []

async def gerar_rotas_Maps(origem_lat: float, origem_long: float, destinos: List[Dict]) -> List[str]:
    """
    Gera URLs para rotas no Google Maps.
    Não usa a API de Directions diretamente, mas sim um link universal do Google Maps
    que abre o aplicativo ou site com a rota pré-definida.
    :param origem_lat: Latitude do ponto de origem.
    :param origem_long: Longitude do ponto de origem.
    :param destinos: Lista de dicionários de destinos (hospitais/médicos) com suas localizações.
    :return: Lista de URLs de rotas.
    """
    rotas = []
    for destino in destinos:
        # Verifica se o dicionário 'destino' contém as informações de localização necessárias.
        if "localizacao" in destino and "lat" in destino["localizacao"] and "lng" in destino["localizacao"]:
            destino_lat = destino["localizacao"]["lat"]
            destino_long = destino["localizacao"]["lng"]
            # Monta a URL de rota.
            # 'google.com/maps/dir/': Caminho para direções.
            # '{origem_lat},{origem_long}': Ponto de partida.
            # 'destination={destino_lat},{destino_long}': Ponto de chegada.
            # 'travelmode=driving': Modo de viagem (carro).
            url = f"https://www.google.com/maps/dir/{origem_lat},{origem_long}/{destino_lat},{destino_long}/data=!3m1!4b1!4m2!4m1!3e0?entry=ttu" # Ajuste para URL de rota mais universal
            rotas.append(url)
        else:
            print(f"Localização inválida para destino: {destino}")
    return rotas


In [3]:
# --- Agente Orquestrador (Função Python que controla o fluxo) ---
# Esta função é o coração do sistema, definindo a sequência em que os agentes
# são chamados e como suas saídas são passadas como entrada para os próximos.
# Ela implementa a lógica de negócios da aplicação de primeiros socorros.

async def orquestrador_primeiros_socorros(mensagem_usuario: str, latitude_usuario: float, longitude_usuario: float):
    """
    Orquestra a interação entre os diferentes agentes de IA para fornecer assistência de primeiros socorros.
    """
    print("⚕️ Iniciando o Sistema de Primeiros Socorros com Agentes ⚕️")

    # 1. Agente de Triagem:
    # O primeiro passo é coletar os sintomas iniciais do usuário.
    print("\n--- 🗣️ Chamando Agente de Triagem... ---\n")
    resposta_triagem = executar_agente(agente_triagem, mensagem_usuario)
    display(to_markdown(f"**Agente de Triagem:** {resposta_triagem}"))
    print("--------------------------------------------------------------")

    # NOVA ETAPA: Coleta de sintomas adicionais do usuário.
    # O orquestrador interage diretamente com o usuário para obter mais detalhes.
    mensagem_adicional_usuario = input("Você sente mais alguma dor ou algo diferente que eu deva saber? (Digite 'não' se não houver): ")

    # 2. Agente de Coleta de Sintomas Adicionais:
    # Consolida a informação inicial e a informação adicional fornecida pelo usuário.
    print("\n--- 🗣️ Chamando Agente de Coleta de Sintomas Adicionais... ---\n")
    sintomas_consolidados = executar_agente(agente_coleta_sintomas_adicionais,
                                            f"Resumo dos sintomas iniciais:\n{resposta_triagem}\n\nSintomas adicionais informados pelo usuário:\n{mensagem_adicional_usuario}")
    display(to_markdown(f"**Sintomas Consolidados:**\n{sintomas_consolidados}"))
    print("--------------------------------------------------------------")

    # 3. Agente de Busca de Primeiros Socorros:
    # Usa a descrição consolidada dos sintomas para buscar informações relevantes na internet.
    print("\n--- 🔍 Chamando Agente de Busca de Primeiros Socorros... ---\n")
    resultados_busca = executar_agente(agente_busca_primeiros_socorros, sintomas_consolidados)
    display(to_markdown(f"**Resultados da Busca:**\n{resultados_busca}"))
    print("--------------------------------------------------------------")

    # 4. Agente de Planejamento de Ações:
    # Com base nos resultados da busca, este agente cria um plano de ações de primeiros socorros.
    print("\n--- 📝 Chamando Agente de Planejamento de Ações... ---\n")
    plano_acoes = executar_agente(agente_planejamento_acoes, resultados_busca)
    display(to_markdown(f"**Plano de Ações:**\n{plano_acoes}"))
    print("--------------------------------------------------------------")

    # 5. Agente de Geração de Orientação Detalhada:
    # Transforma o plano de ações em instruções detalhadas e fáceis de seguir.
    print("\n--- 💡 Chamando Agente de Geração de Orientação Detalhada... ---\n")
    orientacao_detalhada = executar_agente(agente_orientacao_detalhada, plano_acoes)
    display(to_markdown(f"**Orientação Detalhada:**\n{orientacao_detalhada}"))
    print("--------------------------------------------------------------")

    # 6. Agente de Localização de Ajuda Médica e Funções de Maps:
    # O agente 'agente_localizacao_ajuda_medica' raciocina sobre a necessidade,
    # mas as funções Python assíncronas 'buscar_hospitais_proximos' e 'buscar_medicos_proximos'
    # executam a busca real usando a API do Google Maps.
    print("\n--- 🏥 Buscando Hospitais e Médicos Próximos... ---\n")
    # Chamada ao agente para raciocinar sobre a necessidade.
    executar_agente(agente_localizacao_ajuda_medica, "Coletando localização do usuário e buscando locais de ajuda.")

    # Execução das funções assíncronas de busca de Maps.
    hospitais = await buscar_hospitais_proximos(latitude_usuario, longitude_usuario)
    medicos = await buscar_medicos_proximos(latitude_usuario, longitude_usuario)

    # Exibição dos resultados dos hospitais.
    print("\n**Hospitais Próximos:**")
    if hospitais:
        for hospital in hospitais:
            display(to_markdown(f"- **{hospital.get('nome')}** ({hospital.get('endereco')}, Avaliação: {hospital.get('rating', 'N/A')})"))
    else:
        display(to_markdown("Não foram encontrados hospitais próximos."))

    # Exibição dos resultados dos médicos.
    print("\n**Médicos Próximos:**")
    if medicos:
        for medico in medicos:
            display(to_markdown(f"- **{medico.get('nome')}** ({medico.get('endereco')}, Avaliação: {medico.get('rating', 'N/A')})"))
    else:
        display(to_markdown("Não foram encontrados médicos próximos."))
    print("--------------------------------------------------------------")

    # 7. Agente de Rota e Navegação e Funções de Maps:
    # Similarmente, o agente 'agente_rota_navegacao' raciocina,
    # e a função Python 'gerar_rotas_Maps' cria os links de navegação.
    print("\n--- 🗺️ Gerando Rotas para Hospitais... ---\n")
    # Chamada ao agente para raciocinar sobre a necessidade.
    executar_agente(agente_rota_navegacao, "Gerando links de rotas para os hospitais encontrados.")

    # Execução da função de geração de rotas.
    rotas_hospitais = await gerar_rotas_Maps(latitude_usuario, longitude_usuario, hospitais)
    if rotas_hospitais:
        for i, rota in enumerate(rotas_hospitais):
            if i < len(hospitais):
                display(to_markdown(f"Rota para **{hospitais[i].get('nome')}**:"))
                display(HTML(f'<a href="{rota}" target="_blank">{rota}</a>')) # Exibe o link clicável
    else:
        display(to_markdown("Não foi possível gerar rotas para os hospitais."))
    print("--------------------------------------------------------------")

    # 8. Agente de Avaliação de Emergência:
    # Avalia a gravidade da situação com base nos sintomas e plano de ações,
    # e faz uma recomendação final sobre quando e como procurar ajuda médica.
    print("\n--- 🚨 Chamando Agente de Avaliação de Emergência... ---\n")
    recomendacao_emergencia = executar_agente(agente_avaliacao_emergencia, f"Descrição dos sintomas: {sintomas_consolidados}\nPlano de ações: {plano_acoes}")
    display(to_markdown(f"**Avaliação de Emergência:**\n{recomendacao_emergencia}"))
    print("--------------------------------------------------------------")

    # --- Montar Resposta Final Consolidada ---
    # Concatena todas as informações importantes geradas pelos agentes
    # e pelas funções auxiliares em uma única resposta final para o usuário.
    resposta_final = f"""
**Sumário de Primeiros Socorros:**
---
{orientacao_detalhada}

**Locais de Ajuda Médica Próximos:**
"""
    if hospitais:
        resposta_final += "**Hospitais:**\n"
        for hospital in hospitais:
            resposta_final += f"- {hospital.get('nome')} ({hospital.get('endereco')})\n"
    else:
        resposta_final += "Nenhum hospital próximo encontrado.\n"

    if medicos:
        resposta_final += "\n**Médicos/Clínicas:**\n"
        for medico in medicos:
            resposta_final += f"- {medico.get('nome')} ({medico.get('endereco')})\n"
    else:
        resposta_final += "Nenhum médico/clínica próximo encontrado.\n"

    resposta_final += "\n**Rotas (Abrir no Google Maps):**\n"
    if rotas_hospitais:
        for i, rota in enumerate(rotas_hospitais):
            if i < len(hospitais):
                # Inclui o nome do hospital para o link de rota correspondente
                resposta_final += f"- Rota para {hospitais[i].get('nome')}: {rota}\n"
    else:
        resposta_final += "Nenhuma rota gerada.\n"

    resposta_final += f"""
**Recomendação Final de Emergência:**
---
{recomendacao_emergencia}
"""
    # Exibe a resposta final formatada em Markdown.
    display(Markdown(f"\n--- 🤖 **RESPOSTA COMPLETA DO SISTEMA** 🤖 ---\n{resposta_final}"))
    print("--------------------------------------------------------------")

# --- Execução Principal ---
# Esta é a função de entrada do programa, onde a interação com o usuário começa.
async def main():
    """
    Função principal para iniciar o fluxo do sistema de primeiros socorros.
    """
    mensagem_usuario = input("❓ Por favor, descreva o que você está sentindo (ex: 'Dor no peito e formigamento no braço esquerdo'): ")

    # Para testes, usando localização fixa. Em um projeto real, você obteria a localização do usuário.
    # Exemplo: Campinas, SP
    latitude_usuario = -22.9056
    longitude_usuario = -47.0608

    # Chama o orquestrador principal com a mensagem e localização do usuário.
    await orquestrador_primeiros_socorros(mensagem_usuario, latitude_usuario, longitude_usuario)

# Garante que a função 'main' seja executada quando o script é iniciado.
# 'await main()' é necessário pois 'main' é uma função assíncrona.
if __name__ == "__main__":
    await main()

❓ Por favor, descreva o que você está sentindo (ex: 'Dor no peito e formigamento no braço esquerdo'): Febre
⚕️ Iniciando o Sistema de Primeiros Socorros com Agentes ⚕️

--- 🗣️ Chamando Agente de Triagem... ---



NameError: name 'executar_agente' is not defined